## Text Mining II. Music Recomendation Based on Moods

In [1]:
import nltk
import spacy
import pandas as pd

# from gensim.models import Word2Vec

from nltk.tree import Tree
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize

from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download([
    "words",
    "punkt",
    "wordnet",
    "punkt_tab",
    "stopwords",
    "maxent_ne_chunker",
    "vader_lexicon",
])

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\dimas\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package vader_lexico

True

##### Data Preparation

In [3]:
FILE_PATH_EMOTION = './dataset/combined_emotion.csv'
FILE_PATH_SONG = './dataset/high_popularity_spotify_data.csv'

In [4]:
df_emotion = pd.read_csv(FILE_PATH_EMOTION)
print(df_emotion.count(), '\n')
print(df_emotion["emotion"].unique(), '\n')
print(df_emotion.head(), '\n')

df_song = pd.read_csv(FILE_PATH_SONG)
print(df_song[["energy", "liveness", "valence", "track_name"]].head())

sentence    422746
emotion     422746
dtype: int64 

['fear' 'sad' 'love' 'joy' 'suprise' 'anger'] 

                                            sentence emotion
0      i just feel really helpless and heavy hearted    fear
1  ive enjoyed being able to slouch about relax a...     sad
2  i gave up my internship with the dmrg and am f...    fear
3                         i dont know i feel so lost     sad
4  i am a kindergarten teacher and i am thoroughl...    fear 

   energy  liveness  valence          track_name
0   0.592     0.122    0.535    Die With A Smile
1   0.507     0.117    0.438  BIRDS OF A FEATHER
2   0.808     0.159    0.372      That’s So True
3   0.910     0.304    0.786               Taste
4   0.783     0.355    0.939                APT.


### Model for Mood Analysis

##### Load & Process Dataset

In [5]:
sentence = df_emotion['sentence']
emotion = df_emotion['emotion']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    sentence,
    emotion,
    test_size=0.2,
    random_state=42
)

##### Compute TF-IDF Features

In [7]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

##### Training Model

In [8]:
model = ComplementNB()
model.fit(
    X_train_tfidf, 
    y_train
)

ComplementNB()

In [9]:
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 86.97%


##### Sample in Training

In [10]:
for sentence, true_label, predicted_label in zip(X_test[:3], y_test[:3], y_pred[:3]):
    print(f"User Input: {sentence}")
    print(f"True Mood: {true_label} | Predicted Mood: {predicted_label}")
    print("-" * 50)

User Input: i feel like i ve been convinced by some indian guys to go horseback riding with the dalton brothers in cappadocia
True Mood: joy | Predicted Mood: joy
--------------------------------------------------
User Input: i love the insular feel of island living how genuinely weird islanders are it seems to be a prerequisite that you have to be a little off kilter to choose living somewhere one can only access by boat and of course i love being surrounded by the ocean
True Mood: fear | Predicted Mood: suprise
--------------------------------------------------
User Input: i have a feeling that she wasnt thrilled
True Mood: joy | Predicted Mood: joy
--------------------------------------------------


### Music Recomendation

##### Mood Mapping

In [11]:
mood_filters = {
    "joy": {"valence": (0.6, 1.0), "energy": (0.5, 1.0), "liveness": (0.2, 1.0)},
    "sad": {"valence": (0.0, 0.4), "energy": (0.0, 0.5), "liveness": (0.0, 0.2)},
    "anger": {"valence": (0.4, 0.7), "energy": (0.7, 1.0), "liveness": (0.2, 1.0)},
    "fear": {"valence": (0.0, 0.4), "energy": (0.4, 0.7), "liveness": (0.3, 1.0)},
    "love": {"valence": (0.6, 1.0), "energy": (0.4, 0.7), "liveness": (0.2, 0.4)},
    "surprise": {"valence": (0.6, 1.0), "energy": (0.7, 1.0), "liveness": (0.3, 1.0)},
}

##### Extracting User Input

In [12]:
nlp = spacy.load("en_core_web_sm")
def extract_entities(text):
    doc = nlp(text)
    spacy.displacy.render(doc, style="ent", jupyter=True)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    return entities

In [13]:
def recommend_song(mood, df, top_n=5):
    if mood not in mood_filters:
        return f"Invalid mood: {mood}"
    
    filters = mood_filters[mood]
    filtered_songs = df[
        (df["valence"].between(*filters["valence"])) &
        (df["energy"].between(*filters["energy"])) &
        (df["liveness"].between(*filters["liveness"]))
    ]
    
    return filtered_songs.sample(
        n=min(top_n, len(filtered_songs)), 
        random_state=42
    )[["track_name", "track_artist"]]

In [14]:
def predict_mood(user_input):
    user_input_tfidf = vectorizer.transform([user_input])
    return model.predict(user_input_tfidf)[0]

### Menu Section

In [15]:
request = "NO REQUEST"
while True:
    print("\nMUSIC RECOMMENDATION APPLICATION BASED ON MOODS")
    print(f"YOUR REQUEST: {request}")
    print(f"YOUR MOOD: {'UNKNOWN' if request == 'NO REQUEST' else predict_mood(request).item()}")
    print("1. WRITE YOUR REQUEST")
    print("2. VIEW MUSIC RECOMMENDATION")
    print("3. VIEW NAMED ENTITIES RECOGNITION")
    print("4. EXIT")
    
    choice = input(">> ")
    
    if choice == "1":
        request = input("Enter your request: ")
    elif choice == "2":
        if request == "NO REQUEST":
            print("No request found. Please write a request first.")
        else:
            mood = predict_mood(request)
            print(f"Predicted Mood: {mood}")
            print("Recommended Songs:")
            print(recommend_song(mood, df_song))
    elif choice == "3":
        if request == "NO REVIEW":
            print("No request found. Please write a request first.")
            input("Press Enter to continue...")
        else:
            print("Named Entities:")
            extract_entities(request)
    elif choice == "4":
        print("Exiting...")
        break
    else:
        print("Invalid choice. Please select a valid option.")



MUSIC RECOMMENDATION APPLICATION BASED ON MOODS
YOUR REQUEST: NO REQUEST
YOUR MOOD: UNKNOWN
1. WRITE YOUR REQUEST
2. VIEW MUSIC RECOMMENDATION
3. VIEW NAMED ENTITIES RECOGNITION
4. EXIT

MUSIC RECOMMENDATION APPLICATION BASED ON MOODS
YOUR REQUEST: i really loving this summer break
YOUR MOOD: love
1. WRITE YOUR REQUEST
2. VIEW MUSIC RECOMMENDATION
3. VIEW NAMED ENTITIES RECOGNITION
4. EXIT
Predicted Mood: love
Recommended Songs:
                 track_name                                       track_artist
995             Ride Wit Me                                   Nelly, City Spud
1426  THE BOX MEDLEY FUNK 2  THE BOX, Mc Brinquedo, MC Cebezinho, MC Tuto, ...
974              No Diggity                    Blackstreet, Dr. Dre, Queen Pen
1661            Petit génie      Jungeli, Imen Es, Alonzo, Abou Debeing, Lossa
942            Tainted Love                                          Soft Cell

MUSIC RECOMMENDATION APPLICATION BASED ON MOODS
YOUR REQUEST: i really loving this summer b


MUSIC RECOMMENDATION APPLICATION BASED ON MOODS
YOUR REQUEST: i really loving this summer break
YOUR MOOD: love
1. WRITE YOUR REQUEST
2. VIEW MUSIC RECOMMENDATION
3. VIEW NAMED ENTITIES RECOGNITION
4. EXIT
Exiting...
